### Natural Gas Consumption Data by sector
- source: https://www.eia.gov/totalenergy/data/browser/csv.php?tbl=T04.03
- move csv file to current working directory and rename it as "NG Consumption.csv"

In [175]:
import pandas as pd
import numpy as np

df = pd.read_csv("NG Consumption.csv")

In [177]:
df['Description'].unique()

array(['Natural Gas Consumed by the Residential Sector',
       'Natural Gas Consumed by the Commercial Sector',
       'Natural Gas Consumed by the Industrial Sector, Lease and Plant Fuel',
       'Natural Gas Consumed by the Other Industrial Sector, CHP',
       'Natural Gas Consumed by the Other Industrial Sector, Non-CHP',
       'Natural Gas Consumed by the Other Industrial Sector, Total',
       'Natural Gas Consumed by the Industrial Sector, Total',
       'Natural Gas Consumed by the Transportation Sector, Pipelines and Distribution',
       'Natural Gas Consumed by the Transportation Sector, Vehicle Fuel',
       'Natural Gas Consumed by the Transportation Sector, Total',
       'Natural Gas Consumed by the Electric Power Sector',
       'Natural Gas Consumption, Total'], dtype=object)

In [4]:
def prep_gretl(q, descr, filename, suffix='', index_freq='MS', start_date = '2005-01-01', end_date = '2020-01-01'):
    
    # Description
    df = q.query(f'Description == "{descr}"').copy()
    
    # Date Index
    df.loc[:,'YYYYMM'] = df['YYYYMM'].astype(str).copy()
    to_drop = df['YYYYMM'].apply(lambda x: x.endswith('13'))
    df = df[~to_drop].copy()
    df.reset_index(inplace=True, drop=True)
    df['YYYYMM'] = pd.to_datetime(df['YYYYMM'], format='%Y%m').copy()
    df.set_index('YYYYMM', inplace=True)
    df.index.freq = index_freq
    
    # Filter
    filtered_mask = (df.index < end_date) & (df.index >= start_date)
    df_filtered = df[filtered_mask]
    
    # Fixing dtype
    df_filtered.loc[:,'Value'] =  df_filtered.loc[:,'Value'].astype(float).copy()
    
    # Exporting
    gretl = df_filtered.loc[:,'Value'].rename('Value_'+suffix)\
                                      .reset_index(drop=True)
    gretl.to_excel(f'{filename}_{suffix}.xls', index=False)
    
    return df_filtered.loc[:,'Value'].rename('Value_'+suffix)

In [180]:
sfx1 = "industrial"
df1_prep = prep_gretl(df, 'Natural Gas Consumed by the Industrial Sector, Total', 'Series1', sfx1)

sfx2 = "transportation"
df2_prep = prep_gretl(df, 'Natural Gas Consumed by the Transportation Sector, Total', 'Series2', sfx2)

sfx3 = "total"
df3_prep = prep_gretl(df, 'Natural Gas Consumption, Total', 'Series3', sfx3)


C:\Users\giuma\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [182]:
df_ensembled = pd.concat([df1_prep, df2_prep, df3_prep], axis = 1)
df_ensembled.to_excel('df_ensembled.xls', index=False)